<a href="https://colab.research.google.com/github/prafulr89/ML_to_predict_SDN_attacks/blob/main/sdn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils


In [ ]:
import pandas as pd

data_cnt = 10000

pve = pd.read_csv('dataset_attack.csv', nrows = data_cnt).drop(['frame.encap_type', 'ip.src', 'ip.dst','frame.protocols'],axis=1)
nve = pd.read_csv('dataset_normal.csv', nrows = data_cnt).drop(['frame.encap_type','ip.src', 'ip.dst','frame.protocols'],axis=1)

In [ ]:
import numpy as ny

X_P= pve.drop(['class'], axis = 1).values
X_N= nve.drop(['class'], axis = 1).values
y_p= pve['class'].replace(to_replace="attack",value="1")
y_n= pve['class'].replace(to_replace="normal",value="0")
X=ny.concatenate((X_P,X_P))
y=ny.concatenate((y_p,y_p))
print(X)
print(y)

In [ ]:
from sklearn.preprocessing import StandardScaler

slr = StandardScaler(copy=True, with_mean=True, with_std=True)
slr.fit(X)
X = slr.transform(X)
print(X.shape)


In [ ]:
col_cnt = X.shape[1]
print(col_cnt)
row_cnt = X.shape[0]
print(row_cnt)
matrix = ny.zeros((row_cnt-col_cnt, 25, col_cnt))

for i in range(row_cnt-col_cnt):
    temp = ny.zeros((25, col_cnt))
    for j in range(i, i + 25 - 1):
        temp[j-i] = X[j]
    matrix[i] = temp
#print(I)
#print(X.shape)
#print(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(matrix, y[25:20000], test_size = 0.2)
y_train=ny.asarray(y_train).astype(ny.int32)

In [ ]:
def create_baseline():
    model = Sequential()
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
    model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
    model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))
    return model


In [ ]:
model = create_baseline()
model.compile(loss='mse', optimizer="rmsprop", metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 40,validation_split=0.2, verbose = 1)